## <font color='seagreen'> Notebook0 Extract data from Masks and Fluorescent images </font>

In this notebook, we will extract the intensity distributions of both the Vybrant DiD dye (CY5 channel) and Msm-DSRed (CY3 channel) intensity of single-bacteria during macrophage infections. We will also extract these intensities for single macrophages. Previosuly, the masks for single bacteria and macrophages where manually annotated using the Napari image viewer. Here we extract the fluorescent image data per single macrophage or bacteria. 

In [ ]:
#loading libaries.
import napari
import os
import numpy as np
from glob import glob
from joblib import load
from functools import partial
from skimage import io,segmentation,data, feature, future
from sklearn.ensemble import RandomForestClassifier
from magicgui import magicgui
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from skimage.measure import regionprops_table
import pandas as pd

***

## Load image paths

Different time points and properties to extract.

In [ ]:
hours=[0,12,24,36,48, 72]
channels=['DIC','DAPI','CY5','CY3']
props=['label','area','centroid','perimeter']
props_2=['label','area','centroid','intensity_max','intensity_min','intensity_mean']

In [ ]:
def  look_up_images(r):
    print(hours)
    #path multidimensional fluorescent images
    path_3D=fr'..\Data\{hours[r]} Hr\{hours[r]} Hr\*.TIF'
    #path DIC images
    path_DICs=fr'..\Data\{hours[r]} Hr\DIC\*.TIF'
    #path Macrophages
    path_Mask=fr'..\Data\{hours[r]} Hr\{hours[r]} Hr\cell masks\*.TIF'
    #path Bactera
    path_sBacteria=fr'..\Data\{hours[r]} Hr\{hours[r]} Hr\single_bacteria\*.TIF'
    
    #get image full filenames
    TDs=glob(path_3D)
    DICs=glob(path_DICs)
    masks=glob(path_Mask)
    sBacterias=glob(path_sBacteria)
    filenames=glob(path_DICs)

    return filenames,masks, DICs, TDs, sBacterias

Check whether Masks and loaded Images are congruent.

In [ ]:
for r in range(0,len(hours)):
    print(r)
    filenames,masks, DICs, TDs, sBacterias=look_up_images(r)
    for mask,filename, sBacteria, DIC in zip(masks, filenames, sBacterias,DICs):
        fig,ax=plt.subplots(1,2)
        try:
            ax[0].imshow(io.imread(mask))
            ax[1].imshow(io.imread(DIC))
        except:
            print(f'{mask}\n{DIC}')
            pass
        plt.show()

In [ ]:
hours=[0,12,24,36,48,72]
CY5_names={'intenistiy_max':'CY5_intenistiy_max','intensity_min':'CY5_intensity_min','intensity_mean':'CY5_intensity_mean'}
CY3_names={'intenistiy_max':'CY3_intenistiy_max','intensity_min':'CY3_intensity_min','intensity_mean':'CY3_intensity_mean'}

***

## Extract fluorescent intensities for single bacteria

extract data for single-cell bacteria.

In [ ]:
results=[]

for r in range(0,len(hours)):
    print(r)
    filenames,masks, DICs, TDs, sBacterias=look_up_images(r)
    counter=0
    for mask,filename, sBacteria, TD, DIC in zip(masks, filenames, sBacterias,TDs,DICs):
        
        #load images
        img = io.imread(TD, plugin='pil')
        CY3= img[:41,...]
        CY5= img[82:123,...]
        
        #load bacterial masks
        bac_mask=io.imread(sBacteria)
        
        #create regionprop frames
        CY3_frame=pd.DataFrame(regionprops_table(bac_mask, CY3, properties=props_2))
        CY5_frame=pd.DataFrame(regionprops_table(bac_mask, CY5, properties=props_2))
        
        #rename columns
        CY5_frame=CY5_frame.rename(columns=CY5_names)
        CY3_frame=CY3_frame.rename(columns=CY3_names)
        
        #merge bacterial columns 
        result = pd.concat([CY5_frame, CY3_frame], axis=1, join='outer')
        result = result.loc[:,~result.columns.duplicated()].copy()
        
        
        #macrophage
        mac_mask=io.imread(mask)
        
        labels=[] 
        for index,row in result.iterrows():
            x=int(row[3])
            y=int(row[4])
            label=mac_mask[x,y]
            labels.append(label)
        
        result['macrophage_label']=labels
        result['time']=hours[r]
        result['img_ID']=counter
        results.append(result)
        
        counter+=1

In [ ]:
big_frame=pd.concat(results)
big_frame=big_frame.reset_index()
#add unique identifier for each bacteria
big_frame['unique_identifiers']=big_frame['label'].astype(str)+'_'+big_frame['macrophage_label'].astype(str)+'_'+big_frame['time'].astype(str)+'_'+big_frame['img_ID'].astype(str)

In [ ]:
#uncomment to overwrite and save data
#big_frame.to_pickle('../Results/final_single_bacteria.pkl')

***

## Extract macrophage fluorescent intensities

Extract Macrophage single-cell properties.

In [ ]:
results=[]

for r in range(0,len(hours)):
    print(r)
    filenames,masks, DICs, TDs, sBacterias=look_up_images(r)
    counter=0
    for mask,filename, sBacteria, TD, DIC in zip(masks, filenames, sBacterias,TDs,DICs):
        
        #load images
        img = io.imread(TD, plugin='pil')
        CY3= img[:41,...]
        CY3= np.amax(CY3[10:30,...],axis=0)
        CY5= img[82:123,...]
        CY5= np.amax(CY5[10:30,...],axis=0)
        
        #read macrophage mask
        mac_mask=io.imread(mask)
        
        #create regionprop frames
        CY3_frame=pd.DataFrame(regionprops_table(mac_mask, CY3, properties=props_2))
        CY5_frame=pd.DataFrame(regionprops_table(mac_mask, CY5, properties=props_2))
        
        #rename columns
        CY5_frame=CY5_frame.rename(columns=CY5_names)
        CY3_frame=CY3_frame.rename(columns=CY3_names)
        
        #merge bacterial columns 
        result = pd.concat([CY5_frame, CY3_frame], axis=1, join='outer')
        result = result.loc[:,~result.columns.duplicated()].copy()
               
        result['time']=hours[r]
        result['img_ID']=counter
        counter+=1
        results.append(result)
mac_frame=pd.concat(results)
mac_frame=mac_frame.reset_index()

Add unique identifier macrophages.

In [ ]:
mac_frame['unique_ID']=mac_frame['label'].astype(str)+'_'+mac_frame['time'].astype(str)+'_'+mac_frame['img_ID'].astype(str)

add identifier for counting bacteria per macrophage in bacterial dataframe.

In [ ]:
big_frame['unique_identifiers_wolabel']=big_frame['macrophage_label'].astype(str)+'_'+big_frame['time'].astype(str)+'_'+big_frame['img_ID'].astype(str)

In [ ]:
filtered=big_frame[big_frame['CY5_intensity_mean']>0]# no threshold is set here
dict_number_bac=dict(filtered['unique_identifiers_wolabel'].value_counts())
bac_numbers=[]
for key in mac_frame.unique_ID:
    value=dict_number_bac.get(key,0)
    bac_numbers.append(value)
mac_frame['bacteria']= bac_numbers

#plot
axis = sns.violinplot(x="time", y=f"bacteria_CY5", data=mac_frame, color=".8", scale='width')
axis=sns.stripplot(x='time', y=f'bacteria_CY5',data=mac_frame,alpha=0.7, s=3)
axis.set_xlabel('time (hours)')
axis.set_title(f'Number of Bacteria per Macrophage')
axis.set_ylabel('Bacteria')
plt.show()

In [ ]:
#uncomment to overwrite and save data
#mac_frame.to_pickle('..\Results\final_macrophages_dataframe.pkl')

***

In the next Notebook (**Notebooks_Data_exploration\Notebook1_Plotting_Fluorescence_Distributions_of_Vybrant DiD-MSm-DsRed_intensities.ipynb**), we will plot the fluorescent distribution of Vybrant DiD dye.